In [1]:
from collections import OrderedDict
from collections import namedtuple

import binascii

import Crypto
import Crypto.Random
from Crypto.Hash import SHA
from Crypto.PublicKey import RSA
from Crypto.Signature import PKCS1_v1_5

import hashlib
import json
from time import time
from urllib.parse import urlparse
from uuid import uuid4

import requests
from flask import Flask, jsonify, request, render_template
# from flask_cors import CORS

# RSA module

RSA is an asymmetric cryptographic function.

In [2]:
key = RSA.generate(2048) # generates a key using RSA with n = 2048 bits
key

<_RSAobj @0x7fb2905a7fd0 n(2048),e,d,p,q,u,private>

The key (RSA instance) should be considered a container for both private and public keys.  
To be more precise, `key.exportKey('DER')` gives the private key while we have to call `key.publickey().exportKey('DER')` to get access to the public key.

In [3]:
bin_hex = binascii.b2a_hex(b'hello')
bin_hex

b'68656c6c6f'

In [4]:
binascii.unhexlify(bin_hex)

b'hello'

In [5]:
# Getting private key:
pbk = key.exportKey('DER')
pbk

b'0\x82\x04\xa3\x02\x01\x00\x02\x82\x01\x01\x00\x9f\\$\x9b}$!ju_\r\x0fQ\xea\xda\xc5\xd05(\xc7*%\x13\x96\xcd=56\xd9$\xb1\xc4)\xc6\x85b"\x17m\xadw\xb7\x95ON\xb5\xbd\x90\xff\xd9\xf0-\xfb\xcb\x810\x9d\x00[vg\x00\x01\xf0\xcb4q\xf6;{}\xa6j\x01j\x93\xb2\xc8\xa2j\xcb \xb18\xc7\x1e24:k\xd2eK\xc0zR7\xcbj.\\}_\x16^\xc2\x92\x8d\xf0Ue\x8aW_\xee\x82\x0e\xb1a\xb8&\xd1QH\xb55\xf6]c\x9d_J\xe0)\xdd\xfe| \xa6bvr\xf5\x06\xf2i\x99\xab3\xf0.\xf9\xfbA\x90\xc0\x08\x8f\x07ln\xc5wm \x05\x90\xd53D\xd0d\rP@0\'\x1d\x80\x13\xe0\xee\x08\xf7\x0e\xa7\xc7\x13\x88\xcd\xa6]\x05F\xa50m\xcd-\xbf\xcf\xfd\xc8\x12\xf0\x1e\x94k\xaa2x\xfaX\xbe\x9c\x0b81\xc1M\x18\x94q\xe7\xa8\xac\x15\x84\xf2uwq]\x82u\x1f\x03\xb7\xda\x86\xef\x14\x16\x1fO\xdao>Z\xbc\xf0:\x11\xa1\xebu\x02\x03\x01\x00\x01\x02\x82\x01\x00F\xe0\xf7\xd3D\xa4\xb6L j\'g\xe1\xfa\x80U\xadV\x87\x8d\x95\xfc)\xabP\x1b\xe28C\xfac*\xa9\x9e\x07\xa3o V\x82\xea\x15xNn\xe6\xbc\xaa\xeaq\'\xb7|\x1f\xb6\x95E\xafk\xfe\xcc\xaf\x1f-\xd7c\xbb0V\xa2\x85oh\x04\xbb?x,n\xb8\xeb!\xdeQ/\x97\t\x

In [6]:
key.publickey()

<_RSAobj @0x7fb2b00db550 n(2048),e>

In [7]:
# Getting public key:
pvk = key.publickey().exportKey('DER')
pvk

b'0\x82\x01"0\r\x06\t*\x86H\x86\xf7\r\x01\x01\x01\x05\x00\x03\x82\x01\x0f\x000\x82\x01\n\x02\x82\x01\x01\x00\x9f\\$\x9b}$!ju_\r\x0fQ\xea\xda\xc5\xd05(\xc7*%\x13\x96\xcd=56\xd9$\xb1\xc4)\xc6\x85b"\x17m\xadw\xb7\x95ON\xb5\xbd\x90\xff\xd9\xf0-\xfb\xcb\x810\x9d\x00[vg\x00\x01\xf0\xcb4q\xf6;{}\xa6j\x01j\x93\xb2\xc8\xa2j\xcb \xb18\xc7\x1e24:k\xd2eK\xc0zR7\xcbj.\\}_\x16^\xc2\x92\x8d\xf0Ue\x8aW_\xee\x82\x0e\xb1a\xb8&\xd1QH\xb55\xf6]c\x9d_J\xe0)\xdd\xfe| \xa6bvr\xf5\x06\xf2i\x99\xab3\xf0.\xf9\xfbA\x90\xc0\x08\x8f\x07ln\xc5wm \x05\x90\xd53D\xd0d\rP@0\'\x1d\x80\x13\xe0\xee\x08\xf7\x0e\xa7\xc7\x13\x88\xcd\xa6]\x05F\xa50m\xcd-\xbf\xcf\xfd\xc8\x12\xf0\x1e\x94k\xaa2x\xfaX\xbe\x9c\x0b81\xc1M\x18\x94q\xe7\xa8\xac\x15\x84\xf2uwq]\x82u\x1f\x03\xb7\xda\x86\xef\x14\x16\x1fO\xdao>Z\xbc\xf0:\x11\xa1\xebu\x02\x03\x01\x00\x01'

# SHA module

SHA is a hashing function.

In [9]:
h = SHA.new()
h

In [10]:
h.hexdigest()

'da39a3ee5e6b4b0d3255bfef95601890afd80709'

In [11]:
type(b'Hello')

bytes

How to use `h.update`:
> Continue hashing of a message by consuming the next chunk of data.
> Repeated calls are equivalent to a single call with the concatenation
of all the arguments

In [12]:
h.update(b'Hello')
h.hexdigest()

'f7ff9e8b7bb2e09b70935a5d785e0cc5d9d0abf0'

So the above code is strictly equivalent to:

In [13]:
SHA.new(b'Hello').hexdigest()

'f7ff9e8b7bb2e09b70935a5d785e0cc5d9d0abf0'